In [ ]:
import pandas as pd

from data_preparation import (
    download_gios_archive,
    split_raw_df_to_metadata_and_measurements,
    download_updated_metadata,
    build_station_code_mapping,
    update_station_names_metadata,
    update_station_names_data,
    extend_metadata_with_station_info,
    combine_metadata_frames,
    combine_data_frames,
)
from statistics_calculation import (
    analyze_raw_df,
    check_timestamps,
    report_nan_runs,
    monthly_avg_with_nan_threshold,
    hourly_to_daily_30d_sma,
    average_by_city,
    count_days_over_threshold,
)
from visualizations import (
    plot_monthly_avg_station_per_year,
    plot_monthly_avg_station_mean_std_per_year,
    plot_monthly_avg_station_comparison,
    plot_monthly_avg_station_std_comparison,
    plot_daily_sma_per_year,
    plot_daily_sma_mean_std_per_year,
    plot_daily_sma_comparison_years,
    plot_city_monthly_averages,
    plot_city_monthly_heatmaps,
    plot_extreme_stations_days_over,
)

# Przygotowanie danych

## Pobieranie surowych danych

Sprawdzam hashe (wpisane jednorazowo) dla pełnej odtwarzalności (reproducibility)

Funkcja jest w pełni parametryzowana - przekazujemy pełny URL archiwum, nazwę pliku w ZIP i opcjonalny hash.

In [ ]:
gios_archive_urls = {
    2014: "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/302",
    2019: "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/322",
    2024: "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/582",
}
gios_pm25_file = {
    2014: "2014_PM2.5_1g.xlsx",
    2019: "2019_PM25_1g.xlsx",
    2024: "2024_PM25_1g.xlsx",
}
gios_archive_sha256 = {
    2014: "8cabcc2118f019d8d1c0998561c01d57eda8c0a4c531cd2158b18522cd1aed27",
    2019: "777bc03c3c6d1ac77bd4353a80b6e064506368d42be19edece60f040c17dba1c",
    2024: "571dfa56866388c2904284ca6029bbf6016af3905b95bcacc5b3b6f6fa2d00e1",
}
gios_metadata_url = "https://powietrze.gios.gov.pl/pjp/archives/downloadFile/622"
gios_metadata_sha256 = "174290b98ceb780c69769806f7f7a6054015cd78ead8ee65746f3fceba66b2ab"

years = [2014, 2019, 2024]
raw_dfs = {
    year: download_gios_archive(
        gios_archive_urls[year],
        gios_pm25_file[year],
        sha256=gios_archive_sha256[year],
    )
    for year in years
}

## Sprawdzenie pobranych danych

In [ ]:
meta_keys = ["Nr", "Kod stacji", "Wskaźnik",
             "Czas uśredniania", "Jednostka", "Kod stanowiska"]

for year in years:
    year_df = raw_dfs[year]
    print(f"Sanity check for raw {year} data:\n")
    print(analyze_raw_df(year_df, meta_keys))
    print(check_timestamps(year_df, meta_keys, 'H'))
    print(f"\n\n")

## Zmiana formatu dataframe'ów przed dalszym procesowaniem

In [ ]:
meta_keys = ["Nr", "Kod stacji", "Wskaźnik",
             "Czas uśredniania", "Jednostka", "Kod stanowiska"]

meta_by_year = {}
data_by_year = {}
for year in years:
    meta_by_year[year], data_by_year[year] = split_raw_df_to_metadata_and_measurements(raw_dfs[year], meta_keys)

In [ ]:
data_by_year[2014]

## Aktualizacja nazw stacji

In [ ]:
updated_metadata_df = download_updated_metadata(
    gios_metadata_url,
    sha256=gios_metadata_sha256,
)

In [ ]:
updated_metadata_df

In [ ]:
code_map = build_station_code_mapping(updated_metadata_df)

In [ ]:
meta_by_year_updated = {
    year: update_station_names_metadata(meta_by_year[year], updated_metadata_df, code_map, label=str(year))
    for year in years
}

In [ ]:
data_by_year_updated = {
    year: update_station_names_data(data_by_year[year], code_map, label=str(year))
    for year in years
}

## Uzupełnienie i rozszerzenie metadanych

In [ ]:
extra_cols = [
    "Typ stacji",
    "Typ obszaru",
    "Rodzaj stacji",
    "Województwo",
    "Miejscowość",
    "WGS84 φ N",
    "WGS84 λ E",
]

meta_by_year_extended = {
    year: extend_metadata_with_station_info(meta_by_year_updated[year], updated_metadata_df, extra_cols, label=str(year))
    for year in years
}

In [ ]:
rename_geo = {
    "WGS84 φ N": "Szerokość geograficzna",
    "WGS84 λ E": "Długość geograficzna",
}

meta_by_year_extended = {year: df.rename(columns=rename_geo) for year, df in meta_by_year_extended.items()}

## Połączenie danych dla różnych lat

Tworzymy połączony zbiór dla niniejszego opracowania

In [ ]:
metadata_combined, diag1 = combine_metadata_frames(list(meta_by_year_extended.values()))

diag1

In [ ]:
data_combined, diag2 = combine_data_frames(list(data_by_year_updated.values()))

diag2

In [ ]:
assert not diag1.isdisjoint(diag2)

## Czyszczenie / ektrapolacja brakujących danych

In [ ]:
report = report_nan_runs(data_combined, top_k=3)

print("Total stations:", report["total_stations"])
print("Stations with any NaN:", report["stations_with_any_nan"])

for station in report["nan_runs"].keys():
    print(station)
    print(report["nan_runs"][station])

**Interpretacja**

Gdańsk i Siedlce mają wielomiesięczne przerwy w pomiarach.

W pozostałych miastach zdarzają się przerwy rzędu kilkunastu dni.

Jako, że mamy ograniczoną ilość danych - nie wyrzucam tych miast, nie uzupełniam luk.

Natomiast musimy pamiętać:
- Dla analizy miesięcznej: zignorować miesiące z większością dni NaN - niewiarygodna średnia
- Dla analizy dni z przekroczeniami: zignorować Gdańsk i Siedlce - wielomiesięczne luki zaniżają liczbę przekroczeń

## Zapisanie kopii Dataframe'ów na dysku

Surowe dane

In [ ]:
for year in years:
    raw_dfs[year].to_csv(f"raw_data/raw_data_{year}.csv.xz")

updated_metadata_df.to_csv("raw_data/raw_updated_metadata_20251122.csv.xz") # Uwaga - nie są to idealnie surowe dane - skasowaliśmy trailing space w nazwie jednej stacji

Oczyszczone dane

In [ ]:
for year in years:
    meta_by_year_extended[year].to_csv(f"cleaned_data/metadata_{year}.csv.xz")

for year in years:
    data_by_year_updated[year].to_csv(f"cleaned_data/data_{year}.csv.xz")

metadata_combined.to_csv("cleaned_data/pd1_dataset_metadata.csv.xz", index=False)
data_combined.to_csv("cleaned_data/pd1_dataset_data.csv.xz")

# Analiza

## Wczytanie danych

In [ ]:
metadata_combined = pd.read_csv("cleaned_data/pd1_dataset_metadata.csv.xz")
data_combined = pd.read_csv("cleaned_data/pd1_dataset_data.csv.xz",index_col=0)

In [ ]:
data_combined.head()

In [ ]:
metadata_combined.head()

## Średnie miesięczne stacji

> Oblicz średnie miesięczne stężenie PM2.5 dla każdej stacji i roku

Średnie miesięczne stężenie dla każdej stacji rozumiem.
Natomiast "i roku" nie rozumiem.

Chodzi o stężenia miesięczne i roczne?

Średnie stężenie w danym roku we wszystkich stacjach?

In [ ]:
monthly_avg_station = monthly_avg_with_nan_threshold(data_combined, max_nan_per_month=24*10)

In [ ]:
monthly_avg_station.head()

In [ ]:
plot_monthly_avg_station_per_year(monthly_avg_station)


In [ ]:
plot_monthly_avg_station_mean_std_per_year(monthly_avg_station)


In [ ]:
plot_monthly_avg_station_comparison(monthly_avg_station)


In [ ]:
plot_monthly_avg_station_std_comparison(monthly_avg_station)


### SMA

In [ ]:
daily_sma = hourly_to_daily_30d_sma(data_combined)
daily_sma.index = pd.to_datetime(daily_sma.index)

In [ ]:
plot_daily_sma_per_year(daily_sma)


In [ ]:
plot_daily_sma_mean_std_per_year(daily_sma)


In [ ]:
plot_daily_sma_comparison_years(daily_sma)


## Warszawa vs Katowice

In [ ]:
city_data_combined = average_by_city(data_combined, metadata_combined)
city_data_combined.head()

In [ ]:
monthly_avg_city = monthly_avg_with_nan_threshold(city_data_combined, max_nan_per_month=24*10)

In [ ]:
monthly_avg_city

In [ ]:
plot_city_monthly_averages(monthly_avg_city, ["Warszawa", "Katowice"], [2014, 2024])

## Heatmapy

In [ ]:
plot_city_monthly_heatmaps(
    monthly_avg_city,
    cities=city_data_combined.columns.to_list(),
    years=[2014, 2019, 2024],
)

## Przekroczenia norm

In [ ]:
EXCLUDE_STATIONS = ["MzSiedKonars", "PmGdaLeczkow"] # wielomiesięczne przerwy w danych
days_over = count_days_over_threshold(data_combined.drop(EXCLUDE_STATIONS, axis=1), 15, years=(2014, 2019, 2024))
days_over

In [ ]:
plot_extreme_stations_days_over(days_over, year_ref=2024,
                                years=(2014, 2019, 2024), n=3)

## Podsumowanie i Interpretacja

Na podstawie danych można stwierdzić, że sytuacja w zakresie stężenia pyłów **PM2.5** się poprawia na przestrzeni lat. Zdecydowanie widać pogorszenie sytuacji w okresie grzewczym, związanym z emisją pyłów przez prywatne ogrzewanie, które wedle mojej wiedzy, jest wysoko-emisje i przede-wszystkim odpowiedzialne za tak zwany smog. Widać tutaj skutek dotacji mających na celu wymienianie pieców cieplnych na nowoczesne.

Dodatkowo, sytuacja jest różna w różnych badanych regionach. Wynika to prawdopodobnie z czynników socjo-ekonomicznych (koszt wymiany pieca, ogrzewanie centralne, brak poczucia potrzeby dbania o środowisko), a także z uwarunkowania geograficznego (tj. faktu że niektóre miejscowości, są *uwięzione* przez uwarunkowanie terenowe, w taki sposób że zimą, podczas zwiększonej emisji, nie ma na tyle silnego wiatru, bądź jego kierunek nie pozwala na *wywianie* zanieczyszczeń)

### Suchy opis

- Progresywny spadek stężeń w kolejnych latach obserwowany w całym kraju - prawdopodobnie efekt zmian w technologiach grzewczych, spadek udziału paliw stałych (+ możliwe, że również łagodniejsze zimy)
- Silny efekt sezonowości - wzrost w miesiącach jesienno-zimowych - Prawdopodobnie wpływ "niskiej emisji" w sezonie grzewczym
- Wysokie zróżnicowanie stacji w miesiącach jesienno-zimowych, latem spadek we wszystkich stacjach do podobnego poziomu - Prawdopodobnie efekty zróżnicowanych lokalnie metod grzewczych, zróżnicowany udział paliw stałych i pieców starszej generacji
- Wspólny punkt odniesienia latem - prawdopodobnie efekt niższego zróżnicowania stałych źródeł zanieczyszczeń, głównie transportu
- Zróżnicowanie geograficzne: mniejsze poziomy w miejscowościach nadmorskich i na północy kraju (Gdańsk, Szczecin), wyróżniające się negatywnie miasta południa kraju (Kraków, Katowice)
- Ostatni wykres z przekroczeniami dziennymi również wskazuje że spadek stężeń występuje zarówno w "najlepszych" jak i "najgorszych" stacjach